In [2]:
import numpy as np
import pandas as pd

%matplotlib inline

## Params

In [3]:
lgb_submit = pd.read_csv('./data/submission/lgb_estim_validation.csv')
tf_submit = pd.read_csv('./data/submission/tf_estim_validation.csv')
item_ensemble_submit = pd.read_csv('./data/submission/ens_estim_validation.csv')
prophet_top_down_submit = pd.read_csv('./data/submission/Prophet_store_dpt_lgb_weights_validation.csv')
wavenet_top_down_submit = pd.read_csv('./data/submission/Wavenet_store_dpt_lgb_weights_validation.csv')
lgb_top_down_submit = pd.read_csv('./data/submission/Lgb_store_dpt_lgb_weights_validation.csv')

for c in range(1,29):
    tf_submit['F%s' % c].clip(0, inplace=True)

In [5]:
true = pd.read_csv('data/raw/sales_train_evaluation.csv')[['id'] + ['d_%s' % c for c in range(1914, 1942)]]
true.columns = ['id'] + ['F%s' % c for c in range(1, 29)]
true['id'] = true['id'].str.replace('evaluation', 'validation')

In [6]:
import functools

errors = functools.reduce(pd.merge,
                 [
                     pd.melt(lgb_submit.query('id.str.contains("validation")'), id_vars='id', value_name='lgb'),
                     pd.melt(tf_submit.query('id.str.contains("validation")'), id_vars='id', value_name='tf'),
                     pd.melt(item_ensemble_submit.query('id.str.contains("validation")'), id_vars='id', value_name='item_ensemble'),
                     pd.melt(prophet_top_down_submit.query('id.str.contains("validation")'), id_vars='id', value_name='prophet_top_down'),
                     pd.melt(wavenet_top_down_submit.query('id.str.contains("validation")'), id_vars='id', value_name='wavenet_top_down'),
                     pd.melt(lgb_top_down_submit.query('id.str.contains("validation")'), id_vars='id', value_name='lgb_top_down'),
                     pd.melt(true, id_vars='id', value_name='true')
                 ]
                )

errors['neutral'] = 0

In [31]:
pd.read_csv('./data/raw/sales_train_validation.csv')

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,2,0,0,0,0,0,1,0,0,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,2,1,0,2,0,1,0,0,1,0
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [30]:
errors['item_id']

,id,variable,lgb,tf,item_ensemble,prophet_top_down,wavenet_top_down,lgb_top_down,true,neutral,store_id,dept_id,ensemble
0,FOODS_1_001_CA_1_validation,F1,0.902496,0.805387,0.867901,0.818743,0.932849,0.913224,2,0,CA_1,FOODS_1,0.930800
1,FOODS_1_001_CA_2_validation,F1,1.032490,0.761247,0.931837,0.889322,0.952191,0.938254,0,0,CA_2,FOODS_1,1.147120
2,FOODS_1_001_CA_3_validation,F1,1.104004,0.915592,1.014287,0.821182,0.743372,0.784331,1,0,CA_3,FOODS_1,1.059330
3,FOODS_1_001_CA_4_validation,F1,0.405805,0.453879,0.410946,0.441737,0.439455,0.443155,0,0,CA_4,FOODS_1,0.459345
4,FOODS_1_001_TX_1_validation,F1,0.196497,0.362163,0.252284,0.166746,0.173612,0.185001,0,0,TX_1,FOODS_1,0.066630
...,...,...,...,...,...,...,...,...,...,...,...,...,...
853715,HOUSEHOLD_2_516_TX_2_validation,F28,0.304857,0.290513,0.298979,0.298861,0.300084,0.298015,1,0,TX_2,HOUSEHOLD_2,0.340825
853716,HOUSEHOLD_2_516_TX_3_validation,F28,0.192413,0.227177,0.218278,0.164024,0.163627,0.161739,2,0,TX_3,HOUSEHOLD_2,0.154313
853717,HOUSEHOLD_2_516_WI_1_validation,F28,0.137674,0.129373,0.169169,0.099798,0.097321,0.105355,0,0,WI_1,HOUSEHOLD_2,0.098223
853718,HOUSEHOLD_2_516_WI_2_validation,F28,0.103721,0.101995,0.137592,0.073798,0.073727,0.083083,0,0,WI_2,HOUSEHOLD_2,0.063078


In [34]:
errors['store_id'] = errors['id'].apply(lambda x: '_'.join(x.split('_')[3:5]))
errors['dept_id'] = errors['id'].apply(lambda x: '_'.join(x.split('_')[0:2]))
errors['item_id'] = errors['id'].apply(lambda x: '_'.join(x.split('_')[0:3]))

In [37]:
list_ensemble = list()
weights_dict = {}
for _, gp in errors.groupby(['item_id']):
    errors_gp = list()
    for m in ['lgb', 'tf', 'item_ensemble', 'prophet_top_down', 'wavenet_top_down', 'lgb_top_down', 'neutral']:
        errors_gp.append(np.sqrt(((gp['true']-gp[m])**2).mean()))
    
    es = errors_gp[:6]
    ps = gp[['lgb', 'tf', 'item_ensemble', 'prophet_top_down', 'wavenet_top_down', 'lgb_top_down']].values
    e0 = errors_gp[-1]
    l=0.0001
    m = len(es)
    n = len(ps)
    X = ps
    pTy = .5 * (n * e0**2 + (X**2).sum(axis=0) - n * np.array(es)**2)
    w = np.linalg.pinv(X.T.dot(X) + l * n * np.eye(m)).dot(pTy)
    weights_dict[_] = w
    
    gp['ensemble'] = gp[['lgb', 'tf', 'item_ensemble', 'prophet_top_down', 'wavenet_top_down', 'lgb_top_down']].values.dot(w)
    list_ensemble.append(gp)

/home/dataexmachina/DataProjects/m5-forts-castors/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [38]:
ensemble = pd.concat(list_ensemble)
for m in ['lgb', 'tf', 'prophet_top_down', 'wavenet_top_down', 'lgb_top_down', 'ensemble']:
    print(m, np.sqrt(((ensemble['true']-ensemble[m])**2).mean()))

lgb 2.088724547932397
tf 2.098036037985314
prophet_top_down 2.113325014023945
wavenet_top_down 2.118072537261426
lgb_top_down 2.1268339606220352
ensemble 1.9171797412032148


In [39]:
ensemble['ensemble'].clip(0, inplace=True)

In [40]:
ensemble = ensemble.pivot_table(index=['id'], columns=['variable'], values=['ensemble']).reset_index()
ensemble.columns = [c[0] if c[1]=='' else c[1] for c in ensemble.columns.tolist()]
ensemble = ensemble[['id'] + ['F%s' % c for c in range(1,29)]]

In [41]:
ensemble

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,0.733058,0.752450,0.649282,0.547549,1.078188,0.741245,0.856504,0.772486,0.892613,...,0.902455,1.251512,1.121733,0.639496,0.713486,0.575783,0.585924,0.903790,0.967895,1.028987
1,FOODS_1_001_CA_2_validation,0.859671,0.760523,0.947730,1.306092,1.003522,0.897305,1.340442,0.585696,0.655923,...,0.620682,1.297404,1.382055,0.910615,0.770516,0.595995,0.769437,0.829102,1.670124,1.013008
2,FOODS_1_001_CA_3_validation,0.985705,0.944150,0.823427,0.862928,1.002589,0.736460,1.089511,0.991722,1.027933,...,0.939975,1.379605,1.425777,0.935945,0.858745,0.840761,0.922615,1.023716,1.150641,1.100766
3,FOODS_1_001_CA_4_validation,0.226345,0.257665,0.219475,0.239802,0.393246,0.316532,0.360224,0.139130,0.245626,...,0.282615,0.358418,0.406337,0.232203,0.259340,0.295512,0.292837,0.310116,0.340537,0.394623
4,FOODS_1_001_TX_1_validation,0.169084,0.177679,0.162682,0.125865,0.123826,0.152607,0.210435,0.552705,0.438844,...,0.232373,0.366434,0.437127,0.363413,0.320133,0.288383,0.248720,0.259910,0.388645,0.384465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,HOUSEHOLD_2_516_TX_2_validation,0.183113,0.212249,0.216150,0.229624,0.292604,0.302545,0.299301,0.200026,0.211368,...,0.280170,0.288102,0.261726,0.200930,0.204022,0.209420,0.209513,0.274361,0.302307,0.261795
30486,HOUSEHOLD_2_516_TX_3_validation,0.180932,0.169932,0.186145,0.184454,0.229791,0.210844,0.205009,0.136301,0.169750,...,0.230576,0.224326,0.203667,0.150148,0.152883,0.164399,0.169450,0.233138,0.256651,0.251815
30487,HOUSEHOLD_2_516_WI_1_validation,0.084778,0.081883,0.078981,0.081024,0.130004,0.110347,0.102205,0.086236,0.083688,...,0.153423,0.170504,0.112790,0.090794,0.088697,0.090436,0.101569,0.135893,0.181003,0.134689
30488,HOUSEHOLD_2_516_WI_2_validation,0.103666,0.098879,0.104912,0.091467,0.107183,0.077473,0.105265,0.079863,0.094322,...,0.096206,0.108023,0.070043,0.080701,0.081650,0.086261,0.110832,0.143454,0.152043,0.108677


In [42]:
ensemble.to_csv('data/submission/netflix_ensemble.csv', index=False)